In [11]:
import numpy as np

# Define a simple environment with deterministic transitions
# For simplicity, let's assume there are 5 states and
# moving from one state to the next gives a reward of 1, with state 4 being terminal

class SimpleEnvironment:
	def __init__(self, num_states=5):
		self.num_states = num_states

	def step(self, state):
		reward = 0
		terminal = False

		if state < self.num_states - 1:
			next_state = state + 1
			reward = 1
		else:
			next_state = state
			terminal = True

		return next_state, reward, terminal

	def reset(self):
		return 0 # Start from state 0


# Define a random policy for the sake of demonstration
def random_policy(state, num_actions=5):
	return np.random.choice(num_actions)


# Monte Carlo Policy Evaluation function
def monte_carlo_policy_evaluation(policy, env, num_episodes, gamma=1.0):
	value_table = np.zeros(env.num_states)
	returns = {state: [] for state in range(env.num_states)}

	for _ in range(num_episodes):
		state = env.reset()
		episode = []
		# Generate an episode
		while True:
			action = policy(state)
			next_state, reward, terminal = env.step(action)
			episode.append((state, reward))
			if terminal:
				break
			state = next_state

		# Calculate the return and update the value table
		G = 0
		for state, reward in reversed(episode):
			G = gamma * G + reward
			returns[state].append(G)
			value_table[state] = np.mean(returns[state])

	return value_table


# Define the number of episodes for MC evaluation
num_episodes = 1000

# Create a simple environment instance
env = SimpleEnvironment(num_states=5)

# Evaluate the policy
v = monte_carlo_policy_evaluation(random_policy, env, num_episodes)

print("The value table is:")
print(v)


The value table is:
[3.792      3.36126224 3.42258065 3.64761905 3.64128257]


# **EXTRA IF WANTED**

In [12]:
import numpy as np

# Define the grid world environment
class GridWorld:
    def __init__(self):
        self.grid_size = (3, 3)
        self.num_actions = 4  # Up, Down, Left, Right
        self.start_state = (0, 0)
        self.goal_state = (2, 2)
        self.actions = [(0, 1), (0, -1), (-1, 0), (1, 0)]  # Right, Left, Up, Down

    def step(self, state, action):
        next_state = (state[0] + action[0], state[1] + action[1])
        if next_state[0] < 0 or next_state[0] >= self.grid_size[0] or \
                next_state[1] < 0 or next_state[1] >= self.grid_size[1]:
            # Out of bounds, stay in the same state
            return state, -1  # Reward of -1 for hitting the wall
        return next_state, 0 if next_state == self.goal_state else -1  # 0 reward at goal, -1 otherwise

# Define a random policy for the grid world
def random_policy(state):
    return np.random.choice(4)  # Randomly choose an action (0 to 3)

# Monte Carlo simulation
def monte_carlo_simulation(env, policy, num_episodes):
    # Initialize Q-values and state visit counts
    Q = np.zeros((env.grid_size[0], env.grid_size[1], env.num_actions))
    N = np.zeros((env.grid_size[0], env.grid_size[1], env.num_actions))

    for _ in range(num_episodes):
        state = env.start_state
        episode = []
        while state != env.goal_state:
            action = policy(state)
            next_state, reward = env.step(state, env.actions[action])
            episode.append((state, action, reward))
            state = next_state

        G = 0  # Return
        for t in reversed(range(len(episode))):
            state, action, reward = episode[t]
            G = reward + G
            N[state[0], state[1], action] += 1
            Q[state[0], state[1], action] += (1 / N[state[0], state[1], action]) * (G - Q[state[0], state[1], action])

    return Q

# Example usage
env = GridWorld()
Q_values = monte_carlo_simulation(env, random_policy, num_episodes=1000)
print("Q-values:")
print(Q_values)


Q-values:
[[[-26.65612648 -28.27508306 -27.03298153 -28.07179487]
  [-25.33041788 -29.3785092  -26.57428571 -22.89859155]
  [-26.84987893 -27.44014963 -25.99020808 -17.0397351 ]]

 [[-23.29737336 -28.25379171 -30.61722488 -25.23346304]
  [-17.28589909 -26.65049751 -27.88847584 -16.55569307]
  [-16.9860835  -23.26145038 -22.7405303    0.        ]]

 [[-16.26377953 -23.88961039 -27.97290323 -24.05064935]
  [  0.         -22.021611   -23.18994413 -16.2732342 ]
  [  0.           0.           0.           0.        ]]]
